In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import necessary modules for Dash components, including maps, tables, and graphs
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import base64

# Import OS routines and plotting tools
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the CRUD class from Grazioso_Salvare module
from Grazioso_Salvare import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Define the username and password for accessing the database
username = "aacuser"
password = "password"

# Initialize the AnimalShelter object using the provided username and password
shelter = AnimalShelter(password, username)

# Retrieve all data from the database and convert it to a pandas DataFrame
data = shelter.read({})
df = pd.DataFrame.from_records(data)

# Drop the '_id' column, which contains non-serializable ObjectID types from MongoDB
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################

# Create a JupyterDash app instance
app = JupyterDash(__name__)

# Encode the logo image for display in the dashboard
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define the layout of the dashboard
app.layout = html.Div([
    # Centered title of the dashboard
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    
    # Centered logo with a link to the SNHU website
    html.Center(html.A(href='https://snhu.edu', children=[
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                 style={'width': '200px', 'height': 'auto'})])),
    
    # Project details displayed as text with no margins between lines
    html.H6("Project 2 - Bradley Wells", style={'margin-top': '0px', 'margin-bottom': '0px'}),
    html.H6("August, 2024", style={'margin-top': '0px', 'margin-bottom': '0px'}),
    html.H6("Professor Reuben Wilson", style={'margin-top': '0px', 'margin-bottom': '0px'}),
    html.Hr(),  # Horizontal line separator
    
    # Dropdown menu for filtering animal breeds
    html.Div([
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset'  # Default value for the dropdown
        )
    ]),
    html.Hr(),  # Horizontal line separator
    
    # Data table displaying animal information
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),  # Data for the table
                         page_size=10,  # Number of rows per page
                         sort_action='native',  # Enable sorting by column headers
                         style_table={'height': '400px', 'overflowY': 'auto'},  # Style settings for table
                         row_selectable='single',  # Allow single row selection
                         selected_rows=[0],  # Select the first row by default
                         style_data_conditional=[  # Style for alternating row colors
                             {
                                 'if': {'row_index': 'odd'},
                                 'backgroundColor': 'rgb(248, 248, 248)'
                             }
                         ],
                         style_header={  # Style for table header
                             'backgroundColor': 'rgb(230, 230, 230)',
                             'fontWeight': 'bold'
                         },
                         ),
    html.Br(),
    html.Hr(),  # Horizontal line separator
    
    # Div container holding the pie chart and map side by side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',  # Placeholder for the pie chart
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',  # Placeholder for the map
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the data table based on the selected filter
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    query = {}
    
    # Apply filters based on the selected rescue animal selection
    if filter_type == 'ALL':
        data = shelter.read({})
    elif filter_type == 'Water Rescue':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesa Bay Retr Mix", "Newfoundland"]},
                 "sex_upon_outcome": "Intact Female",
                 "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
        data = shelter.read(query)
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute",
                                   "Old English Sheepdog","Siberian Husky", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
        data = shelter.read(query)
    elif filter_type == 'Disaster Rescue or Individual Tracking':
        query = {"breed": {"$in": ["Doberman Pinsch", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
        data = shelter.read(query)
    elif filter_type == 'RESET':
        data = shelter.read({})
    else:
        data = shelter.read({})
    
    # Convert the data to a pandas DataFrame and remove the '_id' column if it exists
    df = pd.DataFrame.from_records(data)
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    
    return df.to_dict('records')  # Return the filtered data as a dictionary

# Callback to update the pie chart based on the filtered data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return []
    
    # Convert the virtual data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Create a pie chart using Plotly Express
    fig = px.pie(dff, names='breed', title='Animal Breed Breakdown')
    
    # Update the pie chart to display the breed name inside each slice
    fig.update_traces(textposition='inside', hovertemplate='%{label}')
    
    # Adjust the text size and uniformity
    fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
    
    return [
        dcc.Graph(figure=fig)  # Return the pie chart as a Dash Graph component
    ]

# Callback to highlight the selected row in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    if selected_rows is None or len(selected_rows) == 0:
        return []
    
    # Return a style dictionary to highlight the selected row
    return [{
        'if': {'row_index': selected_rows[0]},
        'backgroundColor': '#D2F3FF',
        'fontWeight': 'bold'
    }]

# Callback to update the map based on the selected row in the data table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or index is None:
        return []
    
    # Convert the virtual data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Select the row index; if none selected, default to the first row
    row = index[0] if index else 0
    
    # Extract the latitude and longitude for the selected animal
    lat, lon = dff.iloc[row, 13], dff.iloc[row, 14]
    
    # Return the map with a marker at the selected location
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row, 4]),  # Tooltip showing the breed
                dl.Popup([
                    html.H3("Animal Name"),
                    html.P(dff.iloc[row, 9])  # Popup showing the animal's name
                ])
            ])
        ])
    ]

# Run the Dash app with debug mode enabled
app.run_server(debug=True)


Connection to MongoDB Successful
Dash app running on http://127.0.0.1:10507/
